In [7]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
from colorama import Fore, Back, Style
import scipy.stats

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.metrics import roc_auc_score, roc_curve

from lightgbm import LGBMClassifier, early_stopping, log_evaluation

np.set_printoptions(linewidth=150)


In [54]:
##### 데이터를 불러옵니다.
# train = pd.read_csv("../input/tabular-playground-series-aug-2022/train.csv")
# test = pd.read_csv("../input/tabular-playground-series-aug-2022/test.csv")
submission = pd.read_csv("../input/tabular-playground-series-aug-2022/sample_submission.csv")

In [9]:
display(train)
display(test)

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,A,80.10,material_7,material_8,9,5,7,8,4,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,1,A,84.89,material_7,material_8,9,5,14,3,3,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,2,A,82.43,material_7,material_8,9,5,12,1,5,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,3,A,101.07,material_7,material_8,9,5,13,2,6,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,4,A,188.06,material_7,material_8,9,5,9,2,8,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,26565,E,158.95,material_7,material_6,6,9,6,16,4,...,NaN,12.177,17.942,10.112,15.795,18.572,16.144,NaN,729.131,0
26566,26566,E,146.02,material_7,material_6,6,9,10,12,8,...,11.242,14.179,20.564,10.234,14.450,14.322,13.146,16.471,853.924,0
26567,26567,E,115.62,material_7,material_6,6,9,1,10,1,...,11.407,16.437,17.476,8.668,15.069,16.599,15.590,14.065,750.364,0
26568,26568,E,106.38,material_7,material_6,6,9,2,9,4,...,11.392,17.064,17.814,14.928,16.273,15.485,13.624,12.865,730.156,0


,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_8,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17
0,26570,F,119.57,material_5,material_6,6,4,6,9,6,...,18.654,10.802,15.909,18.070,13.772,13.659,16.825,13.742,17.710,634.612
1,26571,F,113.51,material_5,material_6,6,4,11,8,0,...,19.368,12.032,13.998,NaN,12.473,17.468,16.708,14.776,14.102,537.037
2,26572,F,112.16,material_5,material_6,6,4,8,12,4,...,17.774,11.743,17.046,18.086,10.907,13.363,15.737,17.065,16.021,658.995
3,26573,F,112.72,material_5,material_6,6,4,8,11,10,...,18.948,11.790,18.165,16.163,10.933,15.501,15.667,12.620,16.111,594.301
4,26574,F,208.00,material_5,material_6,6,4,14,16,8,...,19.141,12.370,14.578,17.849,11.941,16.070,16.183,13.324,17.150,801.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20770,47340,I,144.74,material_7,material_5,9,5,0,4,9,...,18.573,11.691,NaN,19.771,11.562,17.246,15.131,15.209,NaN,696.466
20771,47341,I,74.53,material_7,material_5,9,5,4,8,7,...,17.917,10.980,16.027,15.694,13.564,15.494,15.296,13.812,16.501,613.249
20772,47342,I,67.73,material_7,material_5,9,5,10,11,2,...,19.630,10.436,16.137,20.612,11.134,16.519,15.525,14.175,17.728,783.349
20773,47343,I,126.15,material_7,material_5,9,5,8,16,11,...,19.575,12.672,15.422,19.496,9.319,15.817,17.403,16.437,15.179,745.210


In [10]:
data = pd.concat([train.reset_index(),test.reset_index()],ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47345 entries, 0 to 47344
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           47345 non-null  int64  
 1   id              47345 non-null  int64  
 2   product_code    47345 non-null  object 
 3   loading         46872 non-null  float64
 4   attribute_0     47345 non-null  object 
 5   attribute_1     47345 non-null  object 
 6   attribute_2     47345 non-null  int64  
 7   attribute_3     47345 non-null  int64  
 8   measurement_0   47345 non-null  int64  
 9   measurement_1   47345 non-null  int64  
 10  measurement_2   47345 non-null  int64  
 11  measurement_3   46635 non-null  float64
 12  measurement_4   46398 non-null  float64
 13  measurement_5   46161 non-null  float64
 14  measurement_6   45925 non-null  float64
 15  measurement_7   45688 non-null  float64
 16  measurement_8   45451 non-null  float64
 17  measurement_9   45214 non-null 

In [11]:
data = data.drop(columns=["index","product_code","id"])
data

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0.0
1,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0.0
2,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0.0
3,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0.0
4,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,144.74,material_7,material_5,9,5,0,4,9,18.465,12.570,...,11.691,NaN,19.771,11.562,17.246,15.131,15.209,NaN,696.466,NaN
47341,74.53,material_7,material_5,9,5,4,8,7,18.900,9.896,...,10.980,16.027,15.694,13.564,15.494,15.296,13.812,16.501,613.249,NaN
47342,67.73,material_7,material_5,9,5,10,11,2,18.656,NaN,...,10.436,16.137,20.612,11.134,16.519,15.525,14.175,17.728,783.349,NaN
47343,126.15,material_7,material_5,9,5,8,16,11,16.536,11.226,...,12.672,15.422,19.496,9.319,15.817,17.403,16.437,15.179,745.210,NaN


In [12]:
data["attribute_2*3"] = data["attribute_2"] * data["attribute_3"]
data

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,attribute_2*3
0,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,...,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0.0,45
1,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,...,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0.0,45
2,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,...,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0.0,45
3,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,...,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0.0,45
4,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,...,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0.0,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,144.74,material_7,material_5,9,5,0,4,9,18.465,12.570,...,NaN,19.771,11.562,17.246,15.131,15.209,NaN,696.466,NaN,45
47341,74.53,material_7,material_5,9,5,4,8,7,18.900,9.896,...,16.027,15.694,13.564,15.494,15.296,13.812,16.501,613.249,NaN,45
47342,67.73,material_7,material_5,9,5,10,11,2,18.656,NaN,...,16.137,20.612,11.134,16.519,15.525,14.175,17.728,783.349,NaN,45
47343,126.15,material_7,material_5,9,5,8,16,11,16.536,11.226,...,15.422,19.496,9.319,15.817,17.403,16.437,15.179,745.210,NaN,45


In [13]:
data["m_3_missing"] = data["measurement_3"].isnull()
data["m_5_missing"] = data["measurement_5"].isnull()
data

,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,...,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure,attribute_2*3,m_3_missing,m_5_missing
0,80.10,material_7,material_8,9,5,7,8,4,18.040,12.518,...,15.193,15.029,NaN,13.034,14.684,764.100,0.0,45,False,False
1,84.89,material_7,material_8,9,5,14,3,3,18.213,11.540,...,11.755,14.732,15.425,14.395,15.631,682.057,0.0,45,False,False
2,82.43,material_7,material_8,9,5,12,1,5,18.057,11.652,...,13.798,16.711,18.631,14.094,17.946,663.376,0.0,45,False,False
3,101.07,material_7,material_8,9,5,13,2,6,17.295,11.188,...,10.020,15.250,15.562,16.154,17.172,826.282,0.0,45,False,False
4,188.06,material_7,material_8,9,5,9,2,8,19.346,12.950,...,12.428,16.182,12.760,13.153,16.412,579.885,0.0,45,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,144.74,material_7,material_5,9,5,0,4,9,18.465,12.570,...,11.562,17.246,15.131,15.209,NaN,696.466,NaN,45,False,False
47341,74.53,material_7,material_5,9,5,4,8,7,18.900,9.896,...,13.564,15.494,15.296,13.812,16.501,613.249,NaN,45,False,False
47342,67.73,material_7,material_5,9,5,10,11,2,18.656,NaN,...,11.134,16.519,15.525,14.175,17.728,783.349,NaN,45,False,False
47343,126.15,material_7,material_5,9,5,8,16,11,16.536,11.226,...,9.319,15.817,17.403,16.437,15.179,745.210,NaN,45,False,False


In [14]:
data['attribute_01'] = data['attribute_0']+data['attribute_1']
data.drop(columns = ['attribute_0', 'attribute_1'], inplace = True)
data_OHE = pd.get_dummies(data,columns=["attribute_01"])
data_OHE

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,failure,attribute_2*3,m_3_missing,m_5_missing,attribute_01_material_5material_5,attribute_01_material_5material_6,attribute_01_material_7material_5,attribute_01_material_7material_6,attribute_01_material_7material_7,attribute_01_material_7material_8
0,80.10,9,5,7,8,4,18.040,12.518,15.748,19.292,...,0.0,45,False,False,0,0,0,0,0,1
1,84.89,9,5,14,3,3,18.213,11.540,17.717,17.893,...,0.0,45,False,False,0,0,0,0,0,1
2,82.43,9,5,12,1,5,18.057,11.652,16.738,18.240,...,0.0,45,False,False,0,0,0,0,0,1
3,101.07,9,5,13,2,6,17.295,11.188,18.576,18.339,...,0.0,45,False,False,0,0,0,0,0,1
4,188.06,9,5,9,2,8,19.346,12.950,16.990,15.746,...,0.0,45,False,False,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47340,144.74,9,5,0,4,9,18.465,12.570,18.146,17.089,...,NaN,45,False,False,0,0,1,0,0,0
47341,74.53,9,5,4,8,7,18.900,9.896,18.288,NaN,...,NaN,45,False,False,0,0,1,0,0,0
47342,67.73,9,5,10,11,2,18.656,NaN,18.242,17.910,...,NaN,45,False,False,0,0,1,0,0,0
47343,126.15,9,5,8,16,11,16.536,11.226,18.144,17.250,...,NaN,45,False,False,0,0,1,0,0,0


In [15]:
train1 = data_OHE.iloc[:len(train)]
test1 = data_OHE.iloc[len(train):]

In [16]:
len(train1) + len(test1) == len(data_OHE)

True

In [17]:
train1.isnull().sum()

loading                               250
attribute_2                             0
attribute_3                             0
measurement_0                           0
measurement_1                           0
measurement_2                           0
measurement_3                         381
measurement_4                         538
measurement_5                         676
measurement_6                         796
measurement_7                         937
measurement_8                        1048
measurement_9                        1227
measurement_10                       1300
measurement_11                       1468
measurement_12                       1601
measurement_13                       1774
measurement_14                       1874
measurement_15                       2009
measurement_16                       2110
measurement_17                       2284
failure                                 0
attribute_2*3                           0
m_3_missing                       

In [18]:
test1.isnull().sum()

loading                                223
attribute_2                              0
attribute_3                              0
measurement_0                            0
measurement_1                            0
measurement_2                            0
measurement_3                          329
measurement_4                          409
measurement_5                          508
measurement_6                          624
measurement_7                          720
measurement_8                          846
measurement_9                          904
measurement_10                        1067
measurement_11                        1136
measurement_12                        1240
measurement_13                        1303
measurement_14                        1440
measurement_15                        1542
measurement_16                        1678
measurement_17                        1740
failure                              20775
attribute_2*3                            0
m_3_missing

In [22]:
from sklearn.preprocessing import StandardScaler

X = train1.drop(columns="failure")
y = train1.failure

scaler = StandardScaler()

tr_X = scaler.fit_transform(X) 

In [21]:
test1 = test1.drop(columns="failure")
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20775 entries, 26570 to 47344
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   loading                            20552 non-null  float64
 1   attribute_2                        20775 non-null  int64  
 2   attribute_3                        20775 non-null  int64  
 3   measurement_0                      20775 non-null  int64  
 4   measurement_1                      20775 non-null  int64  
 5   measurement_2                      20775 non-null  int64  
 6   measurement_3                      20446 non-null  float64
 7   measurement_4                      20366 non-null  float64
 8   measurement_5                      20267 non-null  float64
 9   measurement_6                      20151 non-null  float64
 10  measurement_7                      20055 non-null  float64
 11  measurement_8                      19929 non-null 

In [23]:
te_X = scaler.transform(test1)

In [27]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=2)
imputed = imputer.fit_transform(tr_X)
tr_X1 = pd.DataFrame(imputed, columns=X.columns)
tr_X1.shape

(26570, 30)

In [28]:
from sklearn.impute import KNNImputer


imputed = imputer.transform(te_X)
te_X1 = pd.DataFrame(imputed, columns=test1.columns)
te_X1.shape

(20775, 30)

In [31]:
tr_X1

,loading,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,measurement_4,measurement_5,measurement_6,...,measurement_17,attribute_2*3,m_3_missing,m_5_missing,attribute_01_material_5material_5,attribute_01_material_5material_6,attribute_01_material_7material_5,attribute_01_material_7material_6,attribute_01_material_7material_7,attribute_01_material_7material_8
0,-1.222832,1.525966,-1.538285,-0.101025,-0.055370,-0.681939,0.248179,0.789117,-1.384796,1.788466,...,0.509571,-0.273292,-0.120615,-0.161575,-0.496233,0.0,-0.488091,-0.501705,0.0,1.202276
1,-1.100103,1.525966,-1.538285,1.599402,-1.246039,-0.984141,0.420975,-0.192746,0.591328,0.383792,...,-0.155814,-0.273292,-0.120615,-0.161575,-0.496233,0.0,-0.488091,-0.501705,0.0,1.202276
2,-1.163133,1.525966,-1.538285,1.113565,-1.722306,-0.379737,0.265159,-0.080304,-0.391214,0.732199,...,-0.307320,-0.273292,-0.120615,-0.161575,-0.496233,0.0,-0.488091,-0.501705,0.0,1.202276
3,-0.685543,1.525966,-1.538285,1.356484,-1.484172,-0.077535,-0.495942,-0.546137,1.453436,0.831601,...,1.013879,-0.273292,-0.120615,-0.161575,-0.496233,0.0,-0.488091,-0.501705,0.0,1.202276
4,1.543297,1.525966,-1.538285,0.384811,-1.484172,0.526868,1.552639,1.222823,-0.138302,-1.771917,...,-0.984448,-0.273292,-0.120615,-0.161575,-0.496233,0.0,-0.488091,-0.501705,0.0,1.202276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26565,0.797447,-0.512321,1.208090,-0.343944,1.849699,-0.681939,-1.488770,1.533044,0.943598,-2.013894,...,0.225966,0.621896,-0.120615,-0.161575,-0.496233,0.0,-0.488091,1.993204,0.0,-0.831756
26566,0.466157,-0.512321,1.208090,0.627729,0.897164,0.526868,-0.248234,0.435726,0.859294,1.573598,...,1.238061,0.621896,-0.120615,-0.161575,-0.496233,0.0,-0.488091,1.993204,0.0,-0.831756
26567,-0.312746,-0.512321,1.208090,-1.558534,0.420897,-1.588544,-2.119025,-0.197766,-0.351069,0.877787,...,0.398169,0.621896,-0.120615,-0.161575,-0.496233,0.0,-0.488091,1.993204,0.0,-0.831756
26568,-0.549491,-0.512321,1.208090,-1.315616,0.182763,-0.681939,0.267157,0.319770,-0.210563,0.592635,...,0.234279,0.621896,-0.120615,-0.161575,-0.496233,0.0,-0.488091,1.993204,0.0,-0.831756


from sklearn.model_selection import train_test_split

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(tr_X1, y, test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(21256, 30) (5314, 30) (21256,) (5314,)


In [33]:
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import CategoricalNB
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

In [34]:
# output['failure'] = (model.predict_proba(x_test)[:, 1]) * 0.8
## training
reg = RandomForestClassifier()
reg2 = ExtraTreesClassifier()
reg3 = LogisticRegression()
reg4 = LGBMClassifier()
reg5 = XGBClassifier()


reg.fit(X_train, y_train)
reg2.fit(X_train, y_train)
reg3.fit(X_train, y_train)
reg4.fit(X_train, y_train)
reg5.fit(X_train, y_train)

pred_train = reg.predict_proba(X_train)[:,1]
pred_train2 = reg2.predict_proba(X_train)[:,1]
pred_train3 = reg3.predict_proba(X_train)[:,1]
pred_train4 = reg4.predict_proba(X_train)[:,1]
pred_train5 = reg5.predict_proba(X_train)[:,1]

pred_val = reg.predict_proba(X_val)[:,1]
pred_val2 = reg2.predict_proba(X_val)[:,1]
pred_val3 = reg3.predict_proba(X_val)[:,1]
pred_val4 = reg4.predict_proba(X_val)[:,1]
pred_val5 = reg5.predict_proba(X_val)[:,1]

mse_train = roc_auc_score(y_train, pred_train)
mse_val = roc_auc_score(y_val, pred_val)
mse_train2 = roc_auc_score(y_train, pred_train2)
mse_val2 = roc_auc_score(y_val, pred_val2)
mse_train3 = roc_auc_score(y_train, pred_train3)
mse_val3 = roc_auc_score(y_val, pred_val3)
mse_train4 = roc_auc_score(y_train, pred_train4)
mse_val4 = roc_auc_score(y_val, pred_val4)
mse_train5 = roc_auc_score(y_train, pred_train5)
mse_val5 = roc_auc_score(y_val, pred_val5)

print("1.  Linear Regression\t, train/val = %.5f, %.5f" % (mse_train, mse_val))
print("2.  Lasso\t\t, train/val = %.5f, %.5f" % (mse_train2, mse_val2))
print("3.  Ridge\t\t, train/val = %.5f, %.5f" % (mse_train3, mse_val3))
print("4.  XGBoost\t\t, train/val = %.5f, %.5f" % (mse_train4, mse_val4))
print("5.  LightGBM\t\t, train/val = %.5f, %.5f" % (mse_train5, mse_val5))

1.  Linear Regression	, train/val = 1.00000, 0.54327
2.  Lasso		, train/val = 1.00000, 0.55868
3.  Ridge		, train/val = 0.59388, 0.58864
4.  XGBoost		, train/val = 0.88529, 0.56676
5.  LightGBM		, train/val = 0.96258, 0.53951


In [ ]:
# 실험
# pred_train = reg.predict_proba(X_train)[:, 1]
# pred_train[:, 1]

In [35]:
tr_X1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26570 entries, 0 to 26569
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   loading                            26570 non-null  float64
 1   attribute_2                        26570 non-null  float64
 2   attribute_3                        26570 non-null  float64
 3   measurement_0                      26570 non-null  float64
 4   measurement_1                      26570 non-null  float64
 5   measurement_2                      26570 non-null  float64
 6   measurement_3                      26570 non-null  float64
 7   measurement_4                      26570 non-null  float64
 8   measurement_5                      26570 non-null  float64
 9   measurement_6                      26570 non-null  float64
 10  measurement_7                      26570 non-null  float64
 11  measurement_8                      26570 non-null  flo

In [49]:
pred3= reg3.predict_proba(te_X1)[:,1]

In [55]:
len(pred3)

20775

In [51]:
len(test1) == len(pred3)

True

In [56]:
submission["failure"] = pred3
submission

,id,failure
0,26570,0.220272
1,26571,0.179703
2,26572,0.191094
3,26573,0.198297
4,26574,0.358666
...,...,...
20770,47340,0.243227
20771,47341,0.169641
20772,47342,0.152550
20773,47343,0.235024


In [57]:
submission.reset_index(drop=True).to_csv("submission.csv", index=False)